In [12]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import datetime
import logging
import os
import requests
import numpy as np
from dotenv import load_dotenv
from MCForecastTools import MCSimulation

%matplotlib inline

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [13]:
load_dotenv()
tradev_id = os.getenv("username")
tradev_secret_key =os.getenv("password")

In [14]:
logging.basicConfig(level=logging.DEBUG)
tv=TvDatafeed(auto_login=True)

In [15]:
my_crypto_portfolio = ['AVAXUSD', 'CRVUSD', 'BTCUSD', 'ETHUSD' ]

In [19]:
df_daily = pd.DataFrame()
exchange = 'BINANCE'

for ticker in my_crypto_portfolio:
    data = tv.get_hist(
        symbol = ticker,
        exchange = exchange,
        interval = Interval.in_daily, n_bars=500
    )
    df_daily = df_daily.append(data)
#df_daily.head()           

DEBUG:root:creating websocket connection
DEBUG:tvDatafeed.main:getting data for BINANCE:AVAXUSD...
DEBUG:root:creating websocket connection
DEBUG:tvDatafeed.main:getting data for BINANCE:CRVUSD...
DEBUG:root:creating websocket connection
DEBUG:tvDatafeed.main:getting data for BINANCE:BTCUSD...
DEBUG:root:creating websocket connection
DEBUG:tvDatafeed.main:getting data for BINANCE:ETHUSD...


In [20]:
avax_df = df_daily.loc[df_daily.symbol=='BINANCE:AVAXUSD']
crv_df = df_daily.loc[df_daily.symbol=='BINANCE:CRVUSD']
btc_df = df_daily.loc[df_daily.symbol=='BINANCE:BTCUSD']
eth_df = df_daily.loc[df_daily.symbol=='BINANCE:ETHUSD']
raw_data = pd.concat([avax_df, crv_df, btc_df, eth_df], axis='columns', join='inner')
#raw_data
# May need ways to reshape data automatically so we wont need to create a new dataframe and concat for bigger portfolios

In [29]:
columns = [('AVAXUSD', 'symbol'), ('AVAXUSD', 'open'), ('AVAXUSD', 'high'), ('AVAXUSD', 'low'), ('AVAXUSD', 'close'), ('AVAXUSD', 'volume'),
           ('CRVUSD', 'symbol'), ('CRVUSD', 'open'), ('CRVUSD', 'high'), ('CRVUSD', 'low'), ('CRVUSD', 'close'), ('CRVUSD', 'volume'), 
           ('BTCUSD', 'symbol'), ('BTCUSD', 'open'), ('BTCUSD', 'high'), ('BTCUSD', 'low'), ('BTCUSD', 'close'), ('BTCUSD', 'volume'),
           ('ETHUSD', 'symbol'), ('ETHUSD', 'open'), ('ETHUSD', 'high'), ('ETHUSD', 'low'), ('ETHUSD', 'close'), ('ETHUSD', 'volume')]

raw_data.columns = pd.MultiIndex.from_tuples(columns)
raw_data.head()
                      

AVAXUSD                                   \
                              symbol       open       high        low   
datetime                                                                
2020-08-13 19:00:00  BINANCE:AVAXUSD  10.288204  10.550115  10.081108   
2020-08-14 19:00:00  BINANCE:AVAXUSD  10.366665  10.603421  10.024046   
2020-08-15 19:00:00  BINANCE:AVAXUSD  10.245701  10.556788  10.046086   
2020-08-16 19:00:00  BINANCE:AVAXUSD  10.440332  10.998430   9.795259   
2020-08-17 19:00:00  BINANCE:AVAXUSD  10.443117  10.783283   9.822277   

                                                      CRVUSD             \
                         close        volume          symbol       open   
datetime                                                                  
2020-08-13 19:00:00  10.368185  24471.148557  BINANCE:CRVUSD  13.034994   
2020-08-14 19:00:00  10.244732  20672.062129  BINANCE:CRVUSD  11.453395   
2020-08-15 19:00:00  10.438594  14102.801151  BINANCE:CRVUSD   6.937035   
2020-08-16 19:00:00  10.442608  43704.098340  BINANCE:CRVUSD   5.801475   
2020-08-17 19:00:00  10.324821  36257.446652  BINANCE:CRVUSD   4.393103   

                                           ...        BTCUSD                \
                          high        low  ...          high           low   
datetime                                   ...                               
2020-08-13 19:00:00  56.385610  10.719625  ...  11939.278000  11652.101300   
2020-08-14 19:00:00  25.175398   4.513163  ...  12035.550650  11719.256990   
2020-08-15 19:00:00   7.791000   5.641692  ...  11978.253966  11705.884760   
2020-08-16 19:00:00   6.165889   4.050896  ...  12532.019395  11795.253523   
2020-08-17 19:00:00   4.526203   3.185538  ...  12536.347154  11833.703958   

                                                       ETHUSD              \
                            close      volume          symbol        open   
datetime                                                                    
2020-08-13 19:00:00  11831.019499  298.583619  BINANCE:ETHUSD  425.125966   
2020-08-14 19:00:00  11887.227777  206.908489  BINANCE:ETHUSD  438.699937   
2020-08-15 19:00:00  11955.051879  142.897895  BINANCE:ETHUSD  432.636649   
2020-08-16 19:00:00  12346.909436  471.174900  BINANCE:ETHUSD  433.984179   
2020-08-17 19:00:00  12007.132621  434.301441  BINANCE:ETHUSD  431.557743   

                                                                        
                           high         low       close         volume  
datetime                                                                
2020-08-13 19:00:00  447.323025  416.965372  438.764294  659174.742557  
2020-08-14 19:00:00  447.954257  418.016504  432.524548  329566.835129  
2020-08-15 19:00:00  438.851213  412.211802  433.911909  260671.751151  
2020-08-16 19:00:00  455.072787  410.836204  431.536701  354238.148340  
2020-08-17 19:00:00  442.017872  410.822089  422.290776  308722.482652  

[5 rows x 24 columns]

In [26]:
MC_even_dist = MCSimulation(
    portfolio_data = raw_data, 
    weights = [.25, .25, .25, .25],
    num_simulation=250,
    num_trading_days = 252*3
)

TypeError: get_level_values() missing 1 required positional argument: 'level'

In [88]:
MC_even_dist.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


C:\Users\Aaron\Desktop\FintechProject\notebook\MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,740,741,742,743,744,745,746,747,748,749
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.953206,1.037988,1.014690,1.059926,0.987470,1.048859,0.910286,1.014023,0.953477,0.923059,...,0.977441,1.027175,1.017952,1.046154,1.068622,1.047511,1.034766,0.994923,1.066661,1.052398
2,0.896576,1.070725,1.041566,1.082119,0.933969,1.098560,0.924406,1.004931,0.978333,0.899937,...,0.961499,1.051566,0.981063,1.102777,1.015016,1.056409,1.077952,1.000516,1.093897,0.974766
3,0.864412,1.195925,1.141939,1.095113,0.941821,1.194388,0.872852,0.992653,0.951947,0.891225,...,0.916414,0.977815,0.986224,1.160628,1.024188,1.153664,1.048305,0.948165,1.163770,0.919944
4,0.902769,1.145017,1.212168,1.139210,0.916277,1.223808,0.934089,0.984952,0.952910,0.880998,...,0.947720,1.075644,0.955966,1.209786,0.974801,1.165836,1.155515,0.919696,1.191552,0.950299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,55.656404,34.558001,10.181763,180.572172,79.335285,65.698226,5.136666,12.736593,7.204639,33.926388,...,40.862164,110.586908,100.333420,202.976826,87.373905,49.947502,43.821501,92.674029,6.669503,55.648844
753,61.633789,32.180176,10.049033,186.948972,82.080336,65.733697,5.529550,13.045253,6.722970,35.038397,...,41.830307,117.574056,97.303254,187.320325,92.203360,49.132915,43.488462,92.853101,6.966961,57.284280
754,64.913981,33.848319,10.203849,189.163675,84.453484,65.456985,5.634597,13.403358,6.550435,35.357005,...,45.950162,120.170296,92.992153,181.846726,97.154454,44.951100,42.735859,93.081243,6.964920,57.248609
755,67.601070,34.610595,10.034273,192.383527,84.103165,68.898737,5.653010,12.615212,5.990305,36.694026,...,47.941872,121.842007,97.673507,188.197054,92.662026,47.385256,43.003174,89.886692,7.013193,61.233691


In [89]:
[(my_por


#tot_portfolio = raw_data
#tot_portfolio.drop(columns = ['open', 'high', 'low', 'volume'], inplace = True)
#tot_portfolio

KeyError: 'open'

In [30]:
raw_data.head()

AVAXUSD                                   \
                              symbol       open       high        low   
datetime                                                                
2020-08-13 19:00:00  BINANCE:AVAXUSD  10.288204  10.550115  10.081108   
2020-08-14 19:00:00  BINANCE:AVAXUSD  10.366665  10.603421  10.024046   
2020-08-15 19:00:00  BINANCE:AVAXUSD  10.245701  10.556788  10.046086   
2020-08-16 19:00:00  BINANCE:AVAXUSD  10.440332  10.998430   9.795259   
2020-08-17 19:00:00  BINANCE:AVAXUSD  10.443117  10.783283   9.822277   

                                                      CRVUSD             \
                         close        volume          symbol       open   
datetime                                                                  
2020-08-13 19:00:00  10.368185  24471.148557  BINANCE:CRVUSD  13.034994   
2020-08-14 19:00:00  10.244732  20672.062129  BINANCE:CRVUSD  11.453395   
2020-08-15 19:00:00  10.438594  14102.801151  BINANCE:CRVUSD   6.937035   
2020-08-16 19:00:00  10.442608  43704.098340  BINANCE:CRVUSD   5.801475   
2020-08-17 19:00:00  10.324821  36257.446652  BINANCE:CRVUSD   4.393103   

                                           ...        BTCUSD                \
                          high        low  ...          high           low   
datetime                                   ...                               
2020-08-13 19:00:00  56.385610  10.719625  ...  11939.278000  11652.101300   
2020-08-14 19:00:00  25.175398   4.513163  ...  12035.550650  11719.256990   
2020-08-15 19:00:00   7.791000   5.641692  ...  11978.253966  11705.884760   
2020-08-16 19:00:00   6.165889   4.050896  ...  12532.019395  11795.253523   
2020-08-17 19:00:00   4.526203   3.185538  ...  12536.347154  11833.703958   

                                                       ETHUSD              \
                            close      volume          symbol        open   
datetime                                                                    
2020-08-13 19:00:00  11831.019499  298.583619  BINANCE:ETHUSD  425.125966   
2020-08-14 19:00:00  11887.227777  206.908489  BINANCE:ETHUSD  438.699937   
2020-08-15 19:00:00  11955.051879  142.897895  BINANCE:ETHUSD  432.636649   
2020-08-16 19:00:00  12346.909436  471.174900  BINANCE:ETHUSD  433.984179   
2020-08-17 19:00:00  12007.132621  434.301441  BINANCE:ETHUSD  431.557743   

                                                                        
                           high         low       close         volume  
datetime                                                                
2020-08-13 19:00:00  447.323025  416.965372  438.764294  659174.742557  
2020-08-14 19:00:00  447.954257  418.016504  432.524548  329566.835129  
2020-08-15 19:00:00  438.851213  412.211802  433.911909  260671.751151  
2020-08-16 19:00:00  455.072787  410.836204  431.536701  354238.148340  
2020-08-17 19:00:00  442.017872  410.822089  422.290776  308722.482652  

[5 rows x 24 columns]

In [37]:
tot_portfolio.columns = ['symbol', 'AVAX Close', 'symbol', 'CRV Close', 'symbol', 'BTC Close', 'symbol', 'ETH Close']
tot_portfolio.drop(columns = ['symbol'], inplace = True)
tot_portfolio

# How would I pull and rename close columns automatically? Concat w/ loop through ticker list that looks for the word close?

,AVAX Close,CRV Close,BTC Close,ETH Close
datetime,,,,
2020-08-13 19:00:00,10.368185,11.455075,11831.019499,438.764294
2020-08-14 19:00:00,10.244732,6.896058,11887.227777,432.524548
2020-08-15 19:00:00,10.438594,5.804084,11955.051879,433.911909
2020-08-16 19:00:00,10.442608,4.392889,12346.909436,431.536701
2020-08-17 19:00:00,10.324821,3.596406,12007.132621,422.290776
...,...,...,...,...
2021-09-11 19:00:00,59.164581,2.423487,46364.649493,3405.773112
2021-09-12 19:00:00,53.185420,2.361848,45299.216758,3286.804083
2021-09-13 19:00:00,51.569750,2.799528,47462.824606,3436.019576


In [38]:
daily_return = tot_portfolio.pct_change()
volatility = daily_return.std()*np.sqrt(252)
daily_return

,AVAX Close,CRV Close,BTC Close,ETH Close
datetime,,,,
2020-08-13 19:00:00,NaN,NaN,NaN,NaN
2020-08-14 19:00:00,-0.011907,-0.397991,0.004751,-0.014221
2020-08-15 19:00:00,0.018923,-0.158348,0.005706,0.003208
2020-08-16 19:00:00,0.000385,-0.243138,0.032778,-0.005474
2020-08-17 19:00:00,-0.011279,-0.181312,-0.027519,-0.021426
...,...,...,...,...
2021-09-11 19:00:00,0.036370,0.068262,0.017787,0.042537
2021-09-12 19:00:00,-0.101060,-0.025434,-0.022979,-0.034932
2021-09-13 19:00:00,-0.030378,0.185312,0.047763,0.045398


In [42]:
correlation = tot_portfolio.corr()
correlation

,AVAX Close,CRV Close,BTC Close,ETH Close
AVAX Close,1.000000,0.575231,0.797757,0.759417
CRV Close,0.575231,1.000000,0.462572,0.419466
BTC Close,0.797757,0.462572,1.000000,0.804992
ETH Close,0.759417,0.419466,0.804992,1.000000
